In [1]:
from distutils.command.config import config
import torch 
from torch import nn 
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader 
from dataloaders import CTDataset
from config_file import config_file
from models.unet_from_scratch import UNet
import time
import tqdm

/ubc/ece/home/ra/other/manmeetp/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#making sure we're working using cude and not cpu
print(torch.cuda.is_available())
print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

True
1.12.0
cuda:0


In [3]:
#defining our loss function (combined loss as Arafat described in the email)
def dice_loss(y_pred, y_true, smooth = 1):

    #difference between flatten vs view??

    y_pred = y_pred.view(-1)
    y_true = y_true.view(-1)
    print("the size of the prediction is " + str(y_pred.size()))
    print("the size of true is " + str(y_true.size()))
    smooth = 1
    intersection = (y_true * y_pred).sum()
    dice = (2 * intersection  + smooth)/(y_true.sum() + y_pred.sum() + smooth)

    return 1 - dice()
    
def combo_loss(y_pred, y_true):
    return nn.CrossEntropyLoss(y_pred, y_true) + dice_loss(y_pred, y_true)

In [4]:
#listing the directories 
img_dir = "/ubc/ece/home/ra/other/manmeetp/CTProject/training_set/128_image_arrays"
annotations = "/ubc/ece/home/ra/other/manmeetp/CTProject/training_set/128_mask_arrays"
trainset = CTDataset(annotations_folder=annotations, img_dir=img_dir)
train_loader = DataLoader(trainset, batch_size = config_file.batch_size, shuffle = True)

#channels is just 1 because grayscale image and classes is 2 since its either affected or not 
model = UNet(input_channels=1, num_classes=2)
model.to(device)
loss = dice_loss

<class 'int'>
<class 'int'>
initialization successful


In [5]:
optimizer = torch.optim.Adam(model.parameters(), config_file.learning_rate)
lambda1 = lambda epoch: 0.65 ** epoch

In [6]:
#ok before I start pulling my hair debugging this, let's print the models summary
# print(model)

In [7]:
#training loop 
print("[INFO] training the network...")
startTime = time.time()
for e in range(config_file.epochs):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalTestLoss = 0
	# loop over the training set
	for (i, (x, y)) in enumerate(train_loader):
		# send the input to the device
		(x, y) = (x.to(device), y.to(device))
		x = x.float()
		y = y.float()
		# perform a forward pass and calculate the training loss
		pred = model(x)
		criterion = loss(pred, y)
		# first, zero out any previously accumulated gradients, then
		# perform backpropagation, and then update model parameters
		optimizer.zero_grad()
		criterion.backward()
		optimizer.step()
		# add the loss to the total training loss so far
		print("epoch number " + str(e) + " current loss " + str(criterion))
		totalTrainLoss += criterion
	
	avgTrainLoss = totalTrainLoss / config_file.epochs
	
	print("average Training loss: " + str(avgTrainLoss))
	
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	endTime - startTime))

[INFO] training the network...
entered forward line 1
first line of forward
printing the type <class 'torch.Tensor'>


/ubc/ece/home/ra/other/manmeetp/CTProject/nn_modules/dataloaders.py:58: RuntimeWarning: invalid value encountered in true_divide
  norm_mask_slice = (mask_slice - np.min(mask_slice)) / (np.max(mask_slice) - np.min(mask_slice))


second line of forward
thrd line
fourth line
fifth line
dimensions after the first convlutions torch.Size([2, 64, 128, 128, 128])
dimension after the second convolution torch.Size([2, 128, 64, 64, 64])
dimension after the third convolution torch.Size([2, 256, 32, 32, 32])
dimension after the fourth convolution torch.Size([2, 512, 16, 16, 16])
dimension after the fifth convolution torch.Size([2, 512, 8, 8, 8])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 6 but got size 5 for tensor number 1 in the list.

In [ ]:
#for evaluation
# switch off autograd
	# with torch.no_grad():
	# 	# set the model in evaluation mode
	# 	unet.eval()
	# 	# loop over the validation set
	# 	for (x, y) in testLoader:
	# 		# send the input to the device
	# 		(x, y) = (x.to(config.DEVICE), y.to(config.DEVICE))
	# 		# make the predictions and calculate the validation loss
	# 		pred = unet(x)
	# 		totalTestLoss += lossFunc(pred, y)
	# calculate the average training and validation loss